- VDS 1005210 - Hammett Rd
- freeway: SR99-N, District 10, Stanislaus County
- Lng=121.133332，Lat=37.746753

# 预处理天气数据
- 提取特征：id、经度、维度、海拔、时间、能见度、干球温度、风速、风向、降雨量、云层情况、天气类型
  - HOURLYSKYCONDITIONS（云层）：ccc:ll-xxx
    - 最多三层，超过三层的部分不做记录；天空的总体状况以第三层为主
    - ccc，云层覆盖率
      - CLR, clear sky
      - FEW, few clouds
      - SCT, scattered clouds(散云)，部分多云
      - BKN, broken clouds(碎云)，多云
      - OVC, overcast(阴天)，阴天
      - VV, obscured sky(遮蔽)，由于烟、雾等，天空不可见
      - 10, partially obscured sky
    - ll
      - 00-08量化云层覆盖率
      - 09对应VV
    - xxx，最低云层高度，单位100ft(50 = 5000ft)
    
  - HOURLYVISIBILITY（能见度）：能识别出物体的最小距离，单位英里
  - HOURLYPRSENTWEATHERTYPE（天气类型）：
    - AU|AW|MW（automated weather sensors | automated stations | manually augmented stations）
    - `-`轻度，`+`重度，``中度
    - 缩写详见PDF
  - DAILYWeather（一天天气类型）
  - HOURLYDRYBULBTEMPC（干球温度）：通常所说的温度，单位摄氏度
  - HOURLYWindSpeed（风速）：单位mph
  - HOURLYWindDirection（风向）：风吹来的方向，360 = true north, 180 = south, 270 = east
  - HOURLYPrecip（降雨量）：单位英寸，微量同T表示

## 文件说明
- '../data/weather'，加州所有County天气文件夹，共56个文件
- '../data/county_stations.json'，每个County对应的station
- '../data/all_weather.csv'，直接合并'../data/weather'
- '../data/all_weather_small.csv'，删除部分字段
- '../data/all_weather_new.csv'，重命名

In [2]:
# 合并文件
import os
from tqdm import tqdm
def excel2df(path, save_path):
    columns_name = ['STATION', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'HOURLYDRYBULBTEMPC', 'HOURLYVISIBILITY', 'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYPrecip', 'HOURLYPRSENTWEATHERTYPE', 'HOURLYSKYCONDITIONS']
    df = pd.DataFrame(columns=columns_name)
    for parent, _, names in os.walk(path):
        for name in tqdm(names):
            df_next = pd.read_csv(os.path.join(parent, name), dtype=object)
            df_next = df_next.loc[:, columns_name]
            df = pd.concat([df, df_next], axis=0, ignore_index=True)
        df.to_csv(save_path, index=None)
        print(df.head())
    return 0
excel2df('../data/weather', '../data/all_weather_small.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:58<00:00,  1.05s/it]


      STATION ELEVATION LATITUDE  LONGITUDE              DATE  \
0  WBAN:23285     119.8  37.6927  -121.8144  2016-01-01 00:53   
1  WBAN:23285     119.8  37.6927  -121.8144  2016-01-01 01:53   
2  WBAN:23285     119.8  37.6927  -121.8144  2016-01-01 02:53   
3  WBAN:23285     119.8  37.6927  -121.8144  2016-01-01 03:53   
4  WBAN:23285     119.8  37.6927  -121.8144  2016-01-01 04:53   

  HOURLYDRYBULBTEMPC HOURLYVISIBILITY HOURLYWindSpeed HOURLYWindDirection  \
0                0.6            10.00               0                 000   
1               -0.6            10.00               0                 000   
2               -0.6            10.00               0                 000   
3               -0.6            10.00               3                 090   
4               -1.1            10.00               0                 000   

  HOURLYPrecip HOURLYPRSENTWEATHERTYPE HOURLYSKYCONDITIONS  
0         0.00                     NaN              CLR:00  
1         0.00          

0

In [1]:
# 路段对应检测站
import os
from tqdm import tqdm
import pandas as pd
import json
dic = {}
for parent, _, names in os.walk('../data/weather'):
    for name in tqdm(names):
        station = pd.read_csv(os.path.join(parent, name), dtype=object).loc[:, 'STATION'].drop_duplicates().tolist()
        dic[name] = station
print(dic)

with open('../data/county_stations.json', 'w') as fp:
    json.dump(dic, fp)

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:45<00:00,  1.23it/s]


{'Alameda.csv': ['WBAN:23285', 'WBAN:23230', 'WBAN:93228'], 'Butte.csv': ['WBAN:93210', 'WBAN:93203'], 'Contra Costa.csv': ['WBAN:23254'], 'Del Norte.csv': ['WBAN:24286'], 'El Dorado.csv': ['WBAN:93230', 'WBAN:00227'], 'Fresno County.csv': ['WBAN:93193'], 'Humboldt.csv': ['WBAN:24283', 'WBAN:24213', 'WBAN:00396'], 'Imperial.csv': ['WBAN:23199', 'WBAN:03144'], 'Inyo.csv': ['WBAN:23157', 'WBAN:53139'], 'Kern.csv': ['WBAN:23155', 'WBAN:00479', 'WBAN:03183', 'WBAN:93104', 'WBAN:53144', 'WBAN:23114'], 'Kings.csv': ['WBAN:53119', 'WBAN:23110'], 'Los Angeles1.csv': ['WBAN:23152', 'WBAN:23191', 'WBAN:03165', 'WBAN:03167'], 'Los Angeles2.csv': ['WBAN:03159', 'WBAN:03180', 'WBAN:93134', 'WBAN:23129'], 'Los Angeles3.csv': ['WBAN:23174', 'WBAN:53130', 'WBAN:23182'], 'Los Angeles4.csv': ['WBAN:23187', 'WBAN:93197', 'WBAN:93117'], 'Los Angeles5.csv': ['WBAN:23130', 'WBAN:03174'], 'Madera.csv': ['WBAN:93242'], 'Marin.csv': ['WBAN:00135'], 'Mariposa.csv': ['WBAN:53150'], 'Mendocino.csv': ['WBAN:23275'

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('../data/all_weather_small.csv', dtype=object, date_parser='DATE')

In [4]:
# 取出['STATION', 'LATITUDE', 'LONGITUDE']
# 删除重复行
# 重新索引
# 替换编号
df_station = df.loc[:, ['STATION', 'LATITUDE', 'LONGITUDE']].drop_duplicates().reset_index(drop=True)
df_station.insert(0, 'station_id', df_station.index)
# df_station
# 
df_index = df_station.loc[:, ['station_id', 'STATION']]
df = pd.merge(df, df_index, on='STATION')
columns_new = df.columns.tolist()[-1:] + df.columns.tolist()[:-1]
df = df.loc[:, columns_new]
df.rename(columns={'STATION': 'name', 
                  'LATITUDE': 'latitude', 
                  'LONGITUDE': 'longitude', 
                  'ELEVATION': 'elevation', 
                  'DATE': 'date',
                  'HOURLYDRYBULBTEMPC': 'temp',
                  'HOURLYVISIBILITY': 'visibility',
                  'HOURLYWindSpeed': 'wind_speed',
                  'HOURLYWindDirection': 'wind_dir',
                  'HOURLYPrecip': 'precip', 
                  'HOURLYPRSENTWEATHERTYPE': 'weather',
                  'HOURLYSKYCONDITIONS': 'sky'}, inplace=True)
df.to_csv('../data/all_weather_new.csv', index=None)
df.head()

,station_id,name,elevation,latitude,longitude,date,temp,visibility,wind_speed,wind_dir,precip,weather,sky
0,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 00:53,0.6,10.00,0,000,0.00,NaN,CLR:00
1,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 01:53,-0.6,10.00,0,000,0.00,NaN,CLR:00
2,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 02:53,-0.6,10.00,0,000,0.00,NaN,CLR:00
3,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 03:53,-0.6,10.00,3,090,0.00,NaN,CLR:00
4,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 04:53,-1.1,10.00,0,000,0.00,NaN,CLR:00


## ==============开始载入文件======================

In [5]:
# 计算经纬度之间距离
from math import radians, cos, sin, acos, sqrt
from datetime import datetime, timedelta
def haversine(lon1, lat1, lon2, lat2):
    def hav(theta):
        return (1 - cos(theta)) / 2.0
    # 转成弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # 计算haversine
    hav_h = hav(abs(lon2 - lon1)) + cos(lon1) * cos(lon2) * hav(abs(lat2 - lat1))
    h = acos(1 - 2*hav_h)
    r = 6371  # 地球半径，6371km
    return h * r

# lon1, lat1 = (-121.13333, 37.746753)
# lon2, lat2 = (-121.2258, 37.8891)
# print(haversine(lon1, lat1, lon2, lat2))

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('../data/all_weather_new.csv', dtype=object, parse_dates=['date'])
log, lat = -121.13333, 37.746753
############
df_station = df.loc[:, ['station_id', 'name', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
# df_station.to_csv('../data/df_station.csv', index=None)
df_station['distance'] = df_station.longitude + '_' + df_station.latitude
df_station['distance'] = df_station['distance'].map(lambda x: haversine(float(x.split('_')[0]), float(x.split('_')[1]), log, lat))
df_station.head()

,station_id,name,latitude,longitude,distance
0,0,WBAN:23285,37.6927,-121.8144,75.796511
1,1,WBAN:23230,37.72139,-122.22083,120.933549
2,2,WBAN:93228,37.6542,-122.115,109.289998
3,3,WBAN:93210,39.49,-121.61833,114.423277
4,4,WBAN:93203,39.8,-121.85,143.427872


In [7]:
print(df.count())
df.head()

station_id    4832500
name          4832500
elevation     4832500
latitude      4832500
longitude     4832500
date          4832500
temp          4709890
visibility    3246831
wind_speed    3547138
wind_dir      3439437
precip        1656780
weather        518384
sky           3311404
dtype: int64


,station_id,name,elevation,latitude,longitude,date,temp,visibility,wind_speed,wind_dir,precip,weather,sky
0,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 00:53:00,0.6,10.00,0,000,0.00,NaN,CLR:00
1,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 01:53:00,-0.6,10.00,0,000,0.00,NaN,CLR:00
2,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 02:53:00,-0.6,10.00,0,000,0.00,NaN,CLR:00
3,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 03:53:00,-0.6,10.00,3,090,0.00,NaN,CLR:00
4,0,WBAN:23285,119.8,37.6927,-121.8144,2016-01-01 04:53:00,-1.1,10.00,0,000,0.00,NaN,CLR:00


In [8]:
print(df_station.loc[df_station.distance.idxmin()]['name'])
print(df_station.loc[df_station.distance.idxmin()]['station_id'])

WBAN:23237
98


In [9]:
def str2float(str_num):
    if isinstance(str_num, (float,int)):
        return str_num
    elif str_num == '*' or str_num == 'T':
        return float('nan')
    else:
        str_num = str_num.replace('s', '')  # 删除's'
        str_num = str_num.replace('V', '')
    return float(str_num)

def sky2str(sky):
    try:
        tmp = sky.split(' ')
        if tmp[-1] == '*':
            res = float('nan')
        elif tmp[-1].isdigit() or tmp[-1][:-1].isdigit():  # 数字末尾可能跟's'
            # 最后一位为数字，表示云层高度
            # 取倒数第二位
            # CLR:00 or CLR:00s
            if tmp[-2][-1] == 's':
                res = tmp[-2][-2]
            else:
                res = tmp[-2][-1]
        else:
            if tmp[-1][-1] == 's':
                res = tmp[-2][-2]
            else:
                res = tmp[-1][-1]
    except (AttributeError, IndexError):
        # AttributeError for nan
        # IndexError for '3'
        res = float('nan')
    return float(res)

## ==============开始处理======================

In [10]:
def str2float(str):
    try:
        num = float(str)
    except ValueError:
        try:
            num = float(str[:-1])
        except ValueError:
            num = float('nan')
    return num

def sky2str(sky):
    try:
        tmp = sky.replace('s', '').split(' ')  # 删除字符串中的's'
    except AttributeError:  # for nan
        return float('nan')
    if '*' in tmp:  # for '*'
        return float('nan')
    elif tmp[-1].isdigit():
        # 最后一位为数字，表示云层高度
        # 取倒数第二位
        # CLR:00
        try:
            res = tmp[-2]
        except IndexError:  # for '3'
            return float('nan')
    else:
        res = tmp[-1]
    return res

def weather2code(weather, code):
    i = {'AU': 0, 'AW': 1, 'MW': 2}.get(code)
    try:
        tmp = weather.replace('s', '').split('|')[i]  # 删除字符串中的's'
    except AttributeError:  # for nan
        return float('nan')
    tmp = tmp.replace('*', '')
    if not tmp:
        return float('nan')
    return tmp.split()

def pre_date(date):
    timestamp = round((date.hour * 60 + date.minute) / 5) * 5
    hour = timestamp // 60
    minute = timestamp - hour * 60
    if hour == 24:
        date = date + timedelta(1)
        return datetime(date.year, date.month, date.day, 23, minute, 0)
    return datetime(date.year, date.month, date.day, hour, minute, 0)

In [108]:
df = pd.read_csv('../data/all_weather_new.csv', dtype=object, parse_dates=['date'])
df = df.loc[(df.station_id == '98') & (df.date.dt.year == 2016)].reset_index(drop=True)

In [109]:
# 温度
df['temp'] = df['temp'].apply(str2float).fillna(method='ffill')
# 能见度
df['visibility'] = df['visibility'].apply(str2float).fillna(method='ffill')
# 风速
df['wind_speed'] = df['wind_speed'].apply(str2float).fillna(method='ffill')
# 风向
df['wind_dir'] = df['wind_dir'].apply(str2float).fillna(method='ffill')
# 降雨量
df['precip'] = df['precip'].apply(str2float)  # 缺失太多，不直接填充
# 云层情况
# 'CLR:00', 'FEW:02', 'SCT:04', 'BKN:07', 'OVC:08', 'VV:09', 'X:10'
df['sky'] = df['sky'].apply(sky2str).fillna(method='ffill')
# 天气类型
df['AU'] = df['weather'].apply(weather2code, code='AU')
df['AW'] = df['weather'].apply(weather2code, code='AW')
df['MW'] = df['weather'].apply(weather2code, code='MW')
del df['weather']
# date
df['datetime'] = df['date'].apply(pre_date)

In [126]:
df_pems = pd.read_csv('./data/SR99_VDS1005210_2016_fill.csv', delimiter=';', parse_dates=True, index_col='datetime')
df_weather = weather_dataframe(log, lat)
df_weather = df_weather[['datetime', 'temp', 'visibility']].set_index('datetime')
df = pd.merge(df_pems, df_weather, left_index=True, right_index=True, how='outer').fillna(method='ffill').fillna(method='bfill')

interval = 5
timesteps = 2
epochs = 1
batch_size = int(60 / interval * 24 * 7)  # 每批处理一周数据

train = df[: '2016-10'].copy()
test = df['2016-11': '2016-12'].copy()
col_names = test.columns
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train.values)
scaled_test = scaler.transform(test.values)
for i in range(len(col_names)):
    train[col_names[i]] = scaled_train[:, i]
    test[col_names[i]] = scaled_test[:, i]

x_train, y_train = multi_series2supervised(train, timesteps=timesteps, size=(1, 8))
x_valid, y_valid = multi_series2supervised(train, timesteps=timesteps, size=(9, 10))
x_test, y_test = multi_series2supervised(test, timesteps=timesteps, size=(11, 12))

In [127]:
x_test = x_test.reshape((x_test.shape[0], timesteps * len(col_names)))

In [128]:
x_test

array([[-1.58855397, -0.51204916,  0.3543122 , -1.54760585, -0.51204916,
         0.3543122 ],
       [-1.54760585, -0.51204916,  0.3543122 , -1.48208886, -0.51204916,
         0.3543122 ],
       [-1.48208886, -0.51204916,  0.3543122 , -1.51484735, -0.51204916,
         0.3543122 ],
       ...,
       [-1.41657186, -1.44196491, -4.01700974, -1.58036435, -1.44196491,
        -4.01700974],
       [-1.58036435, -1.44196491, -4.01700974, -1.52303698, -1.44196491,
        -4.01700974],
       [-1.52303698, -1.44196491, -4.01700974, -1.5639851 , -1.44196491,
        -4.01700974]])